# Session 18 🐍

☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️☀️

***

# 145. FastAPI
FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+ based on standard Python type hints.

***

# 146. Core Features
- High Performance: One of the fastest Python frameworks available, rivaling NodeJS and Go (thanks to Starlette and Pydantic)
- Fast to Code: Increases developer speed by 200-300%
- Type Hints: Built-in support with Python's type hints
- Automatic Docs: Interactive API documentation (Swagger UI and ReDoc)
- Standards-based: Based on (and fully compatible with) OpenAPI and JSON Schema

***

# 147. Key Components

***

## 147-1. Path Parameters

In [1]:
@app.get("/items/{item_id}")
def read_item(item_id: int):  # Type conversion and validation
    return {"item_id": item_id}

***

## 147-2. Query Parameters

In [ ]:
@app.get("/items/")
def read_items(skip: int = 0, limit: int = 10):  # Default values
    return {"skip": skip, "limit": limit}

***

## 147-3. Request Body

In [2]:
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: str = None
    price: float
    tax: float = None

@app.post("/items/")
def create_item(item: Item):  # Automatic JSON parsing
    return item

***

## 147-4. Response Model

In [ ]:
@app.post("/items/", response_model=Item)
def create_item(item: Item):  # Filters response data
    return item

***

## 147-5. Status Codes

In [ ]:
from fastapi import status

@app.post("/items/", status_code=status.HTTP_201_CREATED)
def create_item(item: Item):
    return item

***

## 147-6. Form Data

In [ ]:
from fastapi import Form

@app.post("/login/")
def login(username: str = Form(...), password: str = Form(...)):
    return {"username": username}

***

## 147-7. File Uploads

In [ ]:
from fastapi import UploadFile, File

@app.post("/upload/")
def upload_file(file: UploadFile = File(...)):
    return {"filename": file.filename}

***

## 147-8. Dependencies

In [ ]:
from fastapi import Depends

def common_parameters(q: str = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}

@app.get("/items/")
def read_items(commons: dict = Depends(common_parameters)):
    return commons

***

## 147-9. Security

In [ ]:
from fastapi.security import OAuth2PasswordBearer

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

@app.get("/items/")
def read_items(token: str = Depends(oauth2_scheme)):
    return {"token": token}

***

## 147-10. Background Tasks

In [ ]:
from fastapi import BackgroundTasks

def write_notification(email: str, message=""):
    # Send email in background
    pass

@app.post("/send-notification/{email}")
def send_notification(email: str, background_tasks: BackgroundTasks):
    background_tasks.add_task(write_notification, email, message="some notification")
    return {"message": "Notification sent in the background"}

***

# 148. Advanced Features

***

## 148-1. Middleware

In [ ]:
from fastapi import Request

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    process_time = time.time() - start_time
    response.headers["X-Process-Time"] = str(process_time)
    return response

***

## 148-2. CORS

In [ ]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

***

## 148-3. WebSockets

In [ ]:
from fastapi import WebSocket

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        await websocket.send_text(f"Message text was: {data}")

***

## 148-4. Testing

In [ ]:
from fastapi.testclient import TestClient

client = TestClient(app)

def test_read_main():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"Hello": "World"}

***

# 149. Deployment Options
- Uvicorn: uvicorn main:app --host 0.0.0.0 --port 80
- Gunicorn with Uvicorn Workers: gunicorn -k uvicorn.workers.UvicornWorker main:app
- Docker:

In [ ]:
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.8
COPY ./app /app

- Cloud Providers: AWS, Google Cloud, Azure, etc.
- Platforms: Heroku, Docker Swarm, Kubernetes, etc.

***

***

# Some Excercises

**1.** Create a FastAPI application with:
- A root endpoint that returns a welcome message
- An endpoint /items/{item_id} that takes a path parameter item_id (integer)
- The same endpoint should also accept an optional query parameter color (string)
- Return JSON with both values

___

**2.** Create an API for a bookstore with:
- A Pydantic model Book with fields: title (str), author (str), year (int), isbn (str, optional)
- A POST endpoint /books/ that accepts a Book in the request body
- Use a response model to return the created book without the ISBN if it wasn't provided
- Add proper status codes

---

**3.** Create a user profile endpoint that:
- Accepts form data with username and password
- Accepts an optional profile picture file upload
- Returns a JSON response with the username and filename if a file was uploaded
- Validate that the password is at least 8 characters long

---

**4.** Implement a simple authentication system:
- Create a dependency that checks for a header X-Token with value "secret"
- Protect two endpoints with this dependency
- One endpoint should be public (no dependency)
- Return 403 if the token is missing or invalid

***

**5.** Create a newsletter endpoint that:
- Accepts email addresses and a message via POST
- Uses a background task to "send" the email (just log it)
- Returns immediately with a success message
- Simulate a slow email sending process (2 second delay) in the background

***

**6.** Add to an existing FastAPI app:
- Middleware that adds a custom header X-Process-Time showing request processing time
- Configure CORS to allow requests from "example.com" and all localhost ports
- Add a simple endpoint to test these features

***

**7.** Create a simple chat application with:
- A WebSocket endpoint /chat
- That maintains a count of connected clients
- Broadcasts messages from any client to all connected clients
- Sends a welcome message when a client connects

***

**8.** For one of your previous exercises:
- Write at least 3 test cases using TestClient
- Create a Dockerfile for the application
- Write a requirements.txt with all dependencies
- Document how to run it locally and in production

***

#                                                        🌞 https://github.com/AI-Planet 🌞